# Forecasting model to predict future stock prices.

##### This project is an attempt to build a model that predicts the future prices of a chosen public company from the S&P 500, using the  library Prophet to forecast time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.

#### Libraries

In [1]:
import pandas as pd
import warnings; 
warnings.simplefilter('ignore')
#Yahoo Finance API
import yahoo_fin.stock_info as yfi
tickers = yfi.tickers_sp500()
import yfinance as yf
# Prophet
from prophet import Prophet
from prophet.plot import plot_plotly
# Interactive plots
import plotly.io as pio
pio.renderers.default = "notebook_connected"



### Functions for this project:

In [2]:
# Reading the data.
def  get_data(ticker):
    stock = yfi.get_data(ticker, start_date = 2022/0o4/0o1, end_date = None, index_as_date = True, interval = "1d")
    stock["date"] = pd.to_datetime(stock.index)
    stock.reset_index(inplace=True)
    stock_data = stock[["close",  "date"]]
    stock_data.columns = ['y', 'ds']
    return stock_data
    
# Create the model 1
def model2(data):
    m = Prophet(daily_seasonality=True)
    # Train the model
    model = m.fit(data)
    future =  model.make_future_dataframe(periods=365)
    forecast = model.predict(future)
    return model, forecast 

# Create the model 2
def prophet(stock_data):
    m = Prophet(interval_width=0.95, daily_seasonality=True)
    model = m.fit(stock_data)
    future = m.make_future_dataframe(periods=100,freq='D')
    forecast = m.predict(future)
    return m, forecast

#### Getting the data from yahoo API.

In [3]:
stock_data = get_data("MSFT")

# Forecasting with Prophet

In [10]:
#Getting the last 20 days to test our forecast.
last_prices = stock_data[len(stock_data)-20:]
last_prices

,y,ds
9098,280.519989,2022-04-18
9099,285.299988,2022-04-19
9100,286.359985,2022-04-20
9101,280.809998,2022-04-21
9102,274.029999,2022-04-22
9103,280.720001,2022-04-25
9104,270.220001,2022-04-26
9105,283.220001,2022-04-27
9106,289.630005,2022-04-28
9107,277.519989,2022-04-29


In [5]:
# Get al the rows ecxept the last 20 to train the model.
data = stock_data[:-20]
data.shape

(9098, 2)

In [11]:
model, prediction =  model2(data)

Initial log joint probability = -504.698
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       22370.9     0.0120524       17232.1      0.3136      0.3136      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       23393.2    0.00925928       2169.97      0.7301      0.7301      251   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       23653.6    0.00510297       6101.73     0.05825           1      365   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       23936.3    0.00904051       1111.19           1           1      479   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       23975.3    0.00399712       784.222           1           1      590   
    Iter      log prob        ||dx||      ||grad||       alpha  

In [12]:
prediction

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,1986-03-13,0.028340,-20.605106,16.064612,0.028340,0.028340,-1.602179,-1.602179,-1.602179,-0.276369,...,-0.025112,-0.025112,-0.025112,-1.300698,-1.300698,-1.300698,0.0,0.0,0.0,-1.573840
1,1986-03-14,0.029229,-19.199771,15.929108,0.029229,0.029229,-1.644751,-1.644751,-1.644751,-0.276369,...,-0.089543,-0.089543,-0.089543,-1.278838,-1.278838,-1.278838,0.0,0.0,0.0,-1.615522
2,1986-03-17,0.031898,-18.982206,16.437360,0.031898,0.031898,-1.459258,-1.459258,-1.459258,-0.276369,...,-0.010383,-0.010383,-0.010383,-1.172506,-1.172506,-1.172506,0.0,0.0,0.0,-1.427360
3,1986-03-18,0.032788,-19.928937,16.169022,0.032788,0.032788,-1.396898,-1.396898,-1.396898,-0.276369,...,0.005616,0.005616,0.005616,-1.126144,-1.126144,-1.126144,0.0,0.0,0.0,-1.364110
4,1986-03-19,0.033677,-18.663083,15.422633,0.033677,0.033677,-1.301814,-1.301814,-1.301814,-0.276369,...,0.050331,0.050331,0.050331,-1.075775,-1.075775,-1.075775,0.0,0.0,0.0,-1.268137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9458,2023-04-10,300.639339,281.985037,319.813807,298.405388,303.436898,-0.630947,-0.630947,-0.630947,-0.276369,...,-0.010383,-0.010383,-0.010383,-0.344194,-0.344194,-0.344194,0.0,0.0,0.0,300.008393
9459,2023-04-11,300.737632,281.145820,319.105006,298.493000,303.551546,-0.622886,-0.622886,-0.622886,-0.276369,...,0.005616,0.005616,0.005616,-0.352133,-0.352133,-0.352133,0.0,0.0,0.0,300.114746
9460,2023-04-12,300.835925,280.514440,318.473002,298.574613,303.672775,-0.588729,-0.588729,-0.588729,-0.276369,...,0.050331,0.050331,0.050331,-0.362690,-0.362690,-0.362690,0.0,0.0,0.0,300.247196
9461,2023-04-13,300.934218,280.495627,319.218212,298.655663,303.794004,-0.677011,-0.677011,-0.677011,-0.276369,...,-0.025112,-0.025112,-0.025112,-0.375530,-0.375530,-0.375530,0.0,0.0,0.0,300.257206


In [8]:
#ploting
plot_plotly(model, prediction)

In [21]:
# Show model prediction for 05/05/2022
d = "2022-05-01"
prediction_day = prediction[prediction.ds == d]["yhat"].values[0]
prediction_day

265.7840573963679

In [23]:
# Show the actual value
real = last_prices[last_prices.ds == d]["y"].values

Invalid date


##### The difference in % from the predicted price and the actual price. 

In [ ]:
change_percent = "{:.1%}".format(((float(prediction_day)-real)/real))
change_percent 


### Second Model

In [ ]:
m, forecast = prophet(stock_data)

In [ ]:
prediction.tail()

In [ ]:
plot1 = m.plot(forecast)

In [ ]:

plt2 = m.plot_components(forecast)

params = ProphetParams(seasonality_mode='multiplicative') # additive mode gives worse results